In [14]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
train_tape = "Z:/rppg/ccnu_datatape_160x36x36_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_160x36x36_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_tscan(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_tscan(train).cache(f'{tmp}/tscan'), to_tf_tscan(valid).cache(f'{tmp}/tscan')
train, valid = to_tf_tscan(train), to_tf_tscan(valid)

In [16]:
ts_can_e2e = TS_CAN_end_to_end(n=20)
ts_can_e2e.compile(optimizer='Nadam', loss='mse')
ts_can_e2e.build(input_shape=(None, 36, 36, 3))
print(f'Flops per frame:{get_flops(ts_can_e2e, input_sig=[tf.TensorSpec([160, 36, 36, 3])])/160:.0f}')

Flops per frame:52158718


In [ ]:
ts_can_e2e.fit(train.shuffle(2000), validation_data=valid, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/TS-CAN_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
ts_can_e2e.load_weights('../weights/TS-CAN_CCNU.h5')

In [17]:
ts_can_e2e.load_weights('../weights/TS-CAN_CCNU.h5')

In [18]:
def cumsum(result, overwrite=True):
    with h5py.File(result, 'a') as f:
        for i, j in f.items():
            if 'diff' not in j.keys():
                j.create_dataset('diff', data=j['predict'])
                j['predict'][...] = detrend(np.cumsum(j['diff']))
            elif overwrite:
                j['predict'][...] = detrend(np.cumsum(j['diff']))

In [11]:
eval_on_dataset(test_set_CCNU, ts_can_e2e, 160, (36, 36), step=1, batch=0, save='../results/TS-CAN_CCNU_CCNU.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/TS-CAN_CCNU_CCNU.h5')
get_metrics('../results/TS-CAN_CCNU_CCNU.h5')

100%|██████████| 179/179 [09:46<00:00,  3.28s/it]


{'Sliding window': {'MAE': 1.223, 'RMSE': 3.526, 'R': 0.93915},
 'Whole video': {'MAE': 0.929, 'RMSE': 2.898, 'R': 0.95286}}

In [8]:
eval_on_dataset(test_set_CCNU_rPPG, ts_can_e2e, 160, (36, 36), step=1, batch=0, save='../results/TS-CAN_CCNU_CCNU_rPPG.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/TS-CAN_CCNU_CCNU_rPPG.h5')
get_metrics('../results/TS-CAN_CCNU_CCNU_rPPG.h5')

{'Sliding window': {'MAE': 1.233, 'RMSE': 3.816, 'R': 0.92194},
 'Whole video': {'MAE': 0.704, 'RMSE': 1.626, 'R': 0.9847}}

In [21]:
eval_on_dataset(test_set_PURE, ts_can_e2e, 160, (36, 36), step=1, batch=0, save='../results/TS-CAN_CCNU_PURE.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/TS-CAN_CCNU_PURE.h5')
get_metrics('../results/TS-CAN_CCNU_PURE.h5')

100%|██████████| 59/59 [00:40<00:00,  1.47it/s]


{'Sliding window': {'MAE': 4.15, 'RMSE': 11.892, 'R': 0.87403},
 'Whole video': {'MAE': 2.122, 'RMSE': 6.671, 'R': 0.95973}}

In [20]:
eval_on_dataset(test_set_UBFC_rPPG2, ts_can_e2e, 160, (36, 36), step=1, batch=0, save='../results/TS-CAN_CCNU_UBFC.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/TS-CAN_CCNU_UBFC.h5')
get_metrics('../results/TS-CAN_CCNU_UBFC.h5')

100%|██████████| 42/42 [00:27<00:00,  1.53it/s]


{'Sliding window': {'MAE': 1.768, 'RMSE': 4.22, 'R': 0.9701},
 'Whole video': {'MAE': 0.986, 'RMSE': 1.436, 'R': 0.99698}}